# **Auction Hunters**
## Exploratory Data Analysis
#### Joshua Higgins | March~ 2023

## What is *Auction Hunters*?
* Auction hunters is a TV show that aired in the USA from 2010-2015. Hosts Allen and Ton travel and bid for abandoned storage units. Each episode shows where they go, what storage units they bid, how much they win them for, and also shows the most exciting items found in each won unit.
* In each episode, they find buyers for the most profitable items in the storage units they bid on and won.
* The show reports how much each item was sold for, allowing the viewer to keep a tally of their profit for each unit.
* In addition to financial data, we also know where they are and thus can monitor location.
* For more information, check out: https://www.imdb.com/title/tt1742340/

<img src="https://m.media-amazon.com/images/M/MV5BNTc4OTE0MzcxOF5BMl5BanBnXkFtZTcwMjQ0NTM0Ng@@._V1_FMjpg_UX558_.jpg" alt="Hosts Allen and Ton of Auction Hunters">

## Objectives
* A
* B
* C

## Methodology

1. Import necessary libraries and set global variables.
2. Scrape episode data from wikipedia.
3. Organise and clean the data.
4. Dive into the data to answer the objectives.

# 1. Imports and Globals

In [1]:
import requests
from bs4 import BeautifulSoup
import pprint
import pandas as pd
import numpy as np
import re
import datetime
import os

In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_Auction_Hunters_episodes"
DATA_DIR = "data"
WEB_DATA_FILENAME = 'auction_hunters_webpage_content.html'
WEBPAGE = f'{DATA_DIR}/{WEB_DATA_FILENAME}'

# 2. Scrape the Data

## 2.1. Download the webpage to avoid multiple requests

In [3]:
# Send an HTTP request to the URL
response = requests.get(URL)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the content with Beautiful Soup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Save the parsed data to a file
    with open(WEBPAGE, 'w', encoding='utf-8') as website_data:
        website_data.write(str(soup))
else:
    print(f">> Status Code: {response.status_code}. Please check settings and try again.")

## 2.2. Load the local copy of the webpage for parsing

In [4]:
with open(WEBPAGE, 'r', encoding='utf-8') as auction_hunters_wiki_website:
    website_read = auction_hunters_wiki_website.read()
    soup = BeautifulSoup(website_read, 'html.parser')

## 2.3. Inspecting the parsed data to find our table of episode information

* Each season's episode information is stored in a table found in the class `wikitable plainrowheaders wikiepisodetable`.
* We can see that there are 5 seasons by calling the length of the find_all of this class.

In [5]:
all_seasons_scrape = soup.find_all(class_=r"wikitable plainrowheaders wikiepisodetable")
number_of_seasons = len(all_seasons_scrape)
print(f">> There are {number_of_seasons} season(s) in the soup! If this is the number you're expecting, continue!")

>> There are 5 season(s) in the soup! If this is the number you're expecting, continue!


## 2.4. Saving the parsed data for each season individually

In [6]:
season_1_data = all_seasons_scrape[0]
season_2_data = all_seasons_scrape[1]
season_3_data = all_seasons_scrape[2]
season_4_data = all_seasons_scrape[3]
season_5_data = all_seasons_scrape[4]
all_seasons_data = [(season_1_data, 1), 
                    (season_2_data, 2), 
                    (season_3_data, 3), 
                    (season_4_data, 4), 
                    (season_5_data, 5)]

## 2.5. Confirm that each season's table contains a full lists of episodes

* In each season's html data, there are 2 classes which contain the data I want to extract.
   * class `vevent` which contains episode, date and location information, and
   * class `expand-child` which contains episode description with cost, sold and profit text.
* There is 1 class `vevent` and `expand-child` for each episode. 
   * This means we can verify that each episode has full information by counting these classes.

In [7]:
def validate_episode_count(season_X):
    """
    This function is designed to be used in a loop. It will take the season data and extract the class counts for vevent and expand_child.
    If this function is successful, you will confirm all the table data is available for each season.
    This function doesn't confirm the total episode counts, so we trust the wikipedia page is complete.
    """
    season_X_data = season_X[0]
    season_X_num = season_X[1]
    
    vevent_count = len(season_X_data.find_all(class_=r'vevent'))
    expand_child_count = len(season_X_data.find_all(class_=r'expand-child'))
    
    print(f">> Season {season_X_num}: Counted {vevent_count} vevent classes and {expand_child_count} expand-child classes.")
    
    if vevent_count == expand_child_count:
        print(f">>>> Class count matches! Season {season_X_num} has {vevent_count} episodes!\n")
    else:
        print(f">> Class count mis-match. Please check your scrape data / class names.\n")


for season in all_seasons_data:
    validate_episode_count(season)

>> Season 1: Counted 8 vevent classes and 8 expand-child classes.
>>>> Class count matches! Season 1 has 8 episodes!

>> Season 2: Counted 27 vevent classes and 27 expand-child classes.
>>>> Class count matches! Season 2 has 27 episodes!

>> Season 3: Counted 26 vevent classes and 26 expand-child classes.
>>>> Class count matches! Season 3 has 26 episodes!

>> Season 4: Counted 26 vevent classes and 26 expand-child classes.
>>>> Class count matches! Season 4 has 26 episodes!

>> Season 5: Counted 20 vevent classes and 20 expand-child classes.
>>>> Class count matches! Season 5 has 20 episodes!



# 3. Extracting Data

## 3.1. Initial Data Extraction

In [8]:
def get_seasons_episodes_data(season_X_data):
    """
    This function extracts the critical data from the html. Namely:
    
    Data:         [episode number, episode title, location, date]
    Descriptions: [episode description]
    Monetary:     [paid, value, profit]
    
    episode_data = [[Data], [Descriptions], [Monetary]]
                 = [[episode number, episode title, location, date], [episode description], [paid, value, profit]]
    
    
    These 3 lists are captured in a single list for each episode. 
    Thus, the return for each season is a list for each epuside containing 3 lists, which will then need to go through some data cleaning later.
    """
    
    episode_data = []
    episode_descriptions = []
    episode_monetary = []

    # Extract the [episode number, episode title, location, date] data:
    episode_data_tags = season_X_data.find_all(class_=r'vevent')
    for episode_data_tag in episode_data_tags:
        episode_data_list = [data_tag.text for data_tag in episode_data_tag.select("td")]
        episode_data.append(episode_data_list)
        
    # Extract the [episode description] data:
    episode_description_tags = season_X_data.find_all(class_=r'expand-child')
    for episode_description_tag in episode_description_tags:
        episode_description_list = [description.text for description in episode_description_tag.select("p")]
        episode_descriptions.append(episode_description_list)
        
        # In the same class, look for the tag 'dd' containing the monetary [paid, value, profit] data: 
        monetary = [money_values.text for money_values in episode_description_tag.find_all('dd')]
        episode_monetary.append(monetary)

    # Make each episode 1 list of 3 lists (data, description, monetary) and return the entire season's data:
    season_X_full_data_combined = [
        [data, desc, monetary]
        for data, desc, monetary in zip(episode_data, episode_descriptions, episode_monetary)
    ]

    return season_X_full_data_combined

In [9]:
season_1_data_extracted = get_seasons_episodes_data(season_1_data)
season_2_data_extracted = get_seasons_episodes_data(season_2_data)
season_3_data_extracted = get_seasons_episodes_data(season_3_data)
season_4_data_extracted = get_seasons_episodes_data(season_4_data)
season_5_data_extracted = get_seasons_episodes_data(season_5_data)
all_seasons_data_extracted = [(season_1_data_extracted, 1), 
                              (season_2_data_extracted, 2), 
                              (season_3_data_extracted, 3), 
                              (season_4_data_extracted, 4), 
                              (season_5_data_extracted, 5)]

### Validate data extraction:

In [10]:
for season_data_extracted in all_seasons_data_extracted:
    print(f"Season {season_data_extracted[1]}:")
    pprint.pprint(season_data_extracted[0])
    print("")

Season 1:
[[['1',
   '"The Wild West"',
   'San Bernardino, California',
   'November\xa09,\xa02010\xa0(2010-11-09)'],
  ['Ton and Allen head to auctions in the desert town of San Bernardino. Ton '
   'scores a deadly 19th century British Pepper-box handgun and tests it at '
   'the gun range. Allen wins a unit for $1 and finds a fully functional '
   'pre-WWI train set.\n'],
  ['$376', '$1,190', '$814']],
 [['2',
   '"The Big Score"',
   'Downtown Los Angeles, California',
   'November\xa09,\xa02010\xa0(2010-11-09)'],
  ['Ton and Allen bid on units in downtown LA and uncover a ‘70s German H&K P7 '
   'pistol and a rare copper cash register.\n'],
  ['$2,025', '$5,850', '$3,825']],
 [['3',
   '"Ton\'s Got a Gun"',
   'Mission Hills, California',
   'November\xa016,\xa02010\xa0(2010-11-16)'],
  ['Allen encounters some old rivals in the Valley.  Ton and Allen uncover a '
   'Depression-era “Art Case” slot machine, a custom minibike and a Wild West '
   '1880s Colt Peacemaker.\n'],
  ['$1,

## 3.2. Cleaning Data and Converting Season Episode List Data to Dictionary

### 3.2.1. Function: Set Data Field Value - Cleaned or Default

In [11]:
def get_field_value(data, default_value):
    """
    This code will replace repetitive try and except clauses from the original code.
    It will first try to generate a clean data value, but if it can't, it will return a default value.
    This function will be run inside a larger function where the default_value will be passed in.
    
    """
    
    try:
        return data
    except:
        return default_value

### 3.2.2. Function: Generate a Unique Season-Episode Number (as a string)

In [12]:
def generate_unique_season_episode_number(season_num, episode_num):
    """
    This code will generate an episode ID, such as season 1 episode 1 = 101, season 1 episode 12 = 112. 
    It will generate this as a string data type.
    """
    
    s_num = str(season_num)
    ep_num = str(episode_num).zfill(2)
    return s_num + ep_num

### 3.2.3. Function: Generate Episode Data Dictionary with Clean or Default Data

In [13]:
def convert_season_episode_data_to_dict(season_X_data, season_num):
    """
    This function is desidned to be run in a loop and depends on 2 other functions above.
    This function takes season data and season number, then processes each field value, cleans it,
    then stores it in a dictionary. 
    Each episode's data ends up in a dictionary.
    In this way, the entire Auction Hunter show data can be compiled into a large data frame.
    """
    
    field_defaults = {
        'episode_num': 0,
        'episode_name': "----No Episode Name",
        'location': "----No Location",
        'air_date': "----No Air Date",
        'description': "----No Description",
        'paid_$': np.nan,
        'sold_$': np.nan,
        'profit_$': np.nan
    }

    season_X_episode_info_list_of_dicts = []
    for episode in season_X_data:
        ep_num = get_field_value(episode[0][0], field_defaults['episode_num'])
        ep_name = get_field_value(episode[0][1].replace('"', ""), field_defaults['episode_name'])
        ep_loc = get_field_value(episode[0][2], field_defaults['location'])
        
        # Try to extract date information
        try:
            ep_date_initial = episode[0][3]
            ep_date_extract = re.findall(r'\d{4}-\d{2}-\d{2}', ep_date_initial)
            ep_date = datetime.datetime.strptime(ep_date_extract[0], '%Y-%m-%d').date()
        except:
            ep_date = field_defaults['air_date']
        
        # Check if the description list is empty
        if episode[1]:
            ep_desc = get_field_value(episode[1][0].replace("\n", "").replace("  ", " "), field_defaults['description'])
        else:
            ep_desc = field_defaults['description']
            
        ep_paid = get_field_value(episode[2][0].replace(",", "").replace("$", ""), field_defaults['paid_$'])
        ep_sold = get_field_value(episode[2][1].replace(",", "").replace("$", ""), field_defaults['sold_$'])
        ep_profit = get_field_value(episode[2][2].replace(",", "").replace("$", ""), field_defaults['profit_$'])
        
        unique_ep_ID = generate_unique_season_episode_number(season_num, ep_num)
            
        ep_dict = {
            'season_num': season_num,
            'episode_num': ep_num,
            'unique_ep_ID': unique_ep_ID,
            'episode_name': ep_name, 
            'location': ep_loc, 
            'air_date': ep_date, 
            'description': ep_desc, 
            'paid_$': ep_paid, 
            'sold_$': ep_sold, 
            'profit_$': ep_profit
        }

        season_X_episode_info_list_of_dicts.append(ep_dict)

    return season_X_episode_info_list_of_dicts

### 3.2.4. Generate each season's cleaned episode information as a dictionary.

In [14]:
season_1_data_dict = convert_season_episode_data_to_dict(all_seasons_data_extracted[0][0], all_seasons_data_extracted[0][1])
season_2_data_dict = convert_season_episode_data_to_dict(all_seasons_data_extracted[1][0], all_seasons_data_extracted[1][1])
season_3_data_dict = convert_season_episode_data_to_dict(all_seasons_data_extracted[2][0], all_seasons_data_extracted[2][1])
season_4_data_dict = convert_season_episode_data_to_dict(all_seasons_data_extracted[3][0], all_seasons_data_extracted[3][1])
season_5_data_dict = convert_season_episode_data_to_dict(all_seasons_data_extracted[4][0], all_seasons_data_extracted[4][1])
all_seasons_data_dict = [(season_1_data_dict, 1), 
                         (season_2_data_dict, 2), 
                         (season_3_data_dict, 3), 
                         (season_4_data_dict, 4), 
                         (season_5_data_dict, 5)]

In [15]:
for season in all_seasons_data_dict:
    pprint.pprint(season[0])

[{'air_date': datetime.date(2010, 11, 9),
  'description': 'Ton and Allen head to auctions in the desert town of San '
                 'Bernardino. Ton scores a deadly 19th century British '
                 'Pepper-box handgun and tests it at the gun range. Allen wins '
                 'a unit for $1 and finds a fully functional pre-WWI train '
                 'set.',
  'episode_name': 'The Wild West',
  'episode_num': '1',
  'location': 'San Bernardino, California',
  'paid_$': '376',
  'profit_$': '814',
  'season_num': 1,
  'sold_$': '1190',
  'unique_ep_ID': '101'},
 {'air_date': datetime.date(2010, 11, 9),
  'description': 'Ton and Allen bid on units in downtown LA and uncover a ‘70s '
                 'German H&K P7 pistol and a rare copper cash register.',
  'episode_name': 'The Big Score',
  'episode_num': '2',
  'location': 'Downtown Los Angeles, California',
  'paid_$': '2025',
  'profit_$': '3825',
  'season_num': 1,
  'sold_$': '5850',
  'unique_ep_ID': '102'},
 {'air_d

## 3.3. Exporting Dictionaries to CSV by Season

In [16]:
def generate_all_season_dicts_to_csv_and_filenames(all_seasons_dicts):
    """
    This function will take the all_seasons_data_dict list containing cleaned season data dictionaries and season number tuples,
      and converting each season into its own csv file AND a csv file of the ENTIRE series data.
    Returns a list of filenames generated for the CSV files.
    
    """
    generated_filenames = []

    for season_dict in all_seasons_dicts:
        season_data = season_dict[0]
        season_df = pd.DataFrame.from_records(season_data)

        season_num = season_dict[1]
        filename = f"{DATA_DIR}/auction_hunters_season_{season_num}_data.csv"

        # Check if the csv file data already exists:
        if os.path.isfile(filename):
            print(f">> '{filename}' already exists. Skipping save operation. Handle existing data, then try again.")
        else:
            season_df.to_csv(filename, index=False)
            generated_filenames.append(filename)
    
    sorted_generated_filenames = sorted(generated_filenames)

    return sorted_generated_filenames

In [17]:
csv_filenames = generate_all_season_dicts_to_csv_and_filenames(all_seasons_data_dict)

## 3.4. Cleaning the data

### 3.4.1. Generate a list of csv files `(update: incorporated into csv generation function above)`

In [18]:
print(csv_filenames)

['data/auction_hunters_season_1_data.csv', 'data/auction_hunters_season_2_data.csv', 'data/auction_hunters_season_3_data.csv', 'data/auction_hunters_season_4_data.csv', 'data/auction_hunters_season_5_data.csv']


### 3.4.2. Function to load the CSVs into a single dataframe

In [19]:
def load_all_csvs_to_df(list_of_CSVs):
    """
    This function will take a list of csv files, convert them into dataframes, then return a joined single dataframe composed of the input data.
    """
    
    # Specifying the data types so that they are consisent for each loaded dataframe.
    column_data_types_cleaning = {
        'season_num': 'int8',
        'episode_num': 'int8',
        'unique_ep_ID': 'int16',
        'episode_name': 'object',
        'location': 'object',
        'air_date': 'object',
        'description': 'object',   
        'paid_$': 'object',
        'sold_$': 'object',
        'profit_$': 'object' 
    }			
    
    # Create a list of dataframe objects
    df_list = []   
    for csv_file in list_of_CSVs:
        df = pd.read_csv(csv_file, dtype=column_data_types_cleaning)
        df_list.append(df)
        
    # Concatenate the dataframes into a single dataframe object.
    # Also, reset the index to avoid concatenation restarting the index at each join.
    combined_df = pd.concat(df_list, axis=0)
    combined_df = combined_df.reset_index(drop=True)
    
    return combined_df

### 3.4.3. Create the full dataframe

In [20]:
# Set Pandas display options so that we can see the entire dataframe.
pd.set_option('display.max_rows', None)     # Shows all rows
pd.set_option('display.max_columns', None)  # Shows all columns
pd.set_option('display.width', None)        # Adjusts the display width

df = load_all_csvs_to_df(csv_filenames)
df

,season_num,episode_num,unique_ep_ID,episode_name,location,air_date,description,paid_$,sold_$,profit_$
0,1,1,101,The Wild West,"San Bernardino, California",2010-11-09,Ton and Allen head to auctions in the desert t...,376,1190,814
1,1,2,102,The Big Score,"Downtown Los Angeles, California",2010-11-09,Ton and Allen bid on units in downtown LA and ...,2025,5850,3825
2,1,3,103,Ton's Got a Gun,"Mission Hills, California",2010-11-16,Allen encounters some old rivals in the Valley...,1900,9450,7550
3,1,4,104,Strat'ed for Cash,"West Adams, Los Angeles",2010-11-23,Allen and Ton venture out to storage auctions ...,975,6200,5225
4,1,5,105,The Real Thing,"Oxnard, California",2010-11-30,"In a small beachtown’s storage unit auction, T...",2450,9400,6950
5,1,6,106,Weapons of Past Destruction,Sunset Boulevard,2010-12-07,Ton and Allen travel to Hollywood and uncover ...,295,1635,1340
6,1,7,107,Gangster Whiskey,"Moreno Valley, California",2010-12-14,"A rare baseball card, a classic jukebox and an...",525,2800,2275
7,1,8,108,Home on the Gun Range,"Corona, California",2010-12-21,A single unit auction yields a 1940s Kissomete...,875,3400,2425
8,2,1,201,Miami Heat,Miami,2011-04-05,"A 1970s NASA countdown clock, a 1960s Delta re...",2800,10640,7840
9,2,2,202,Quadzilla Attacks!,"Playa Vista, Los Angeles",2011-04-05,"An 1880s Palm-Squeezer pistol, a Western Elect...",1250,4990,3740


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   season_num    107 non-null    int8  
 1   episode_num   107 non-null    int8  
 2   unique_ep_ID  107 non-null    int16 
 3   episode_name  107 non-null    object
 4   location      106 non-null    object
 5   air_date      107 non-null    object
 6   description   107 non-null    object
 7   paid_$        107 non-null    object
 8   sold_$        107 non-null    object
 9   profit_$      107 non-null    object
dtypes: int16(1), int8(2), object(7)
memory usage: 6.4+ KB


`The dataframe appears structured. There is just 1 Null, however default values could be masking them. Datatypes are OK as we are still cleaning.
107 rows of data`

## 3.5. There is some basic cleaning to do on the base dataset

1. Remove the text from the Paid, Sold and Profit fields and cast datatypes.
3. Cast air_date datatype correctly.
2. Ensure all fields are filled. e.g:
   * episode 514: Location
   * episode 401: Description

In [22]:
df_cleaning = df.copy()

### 3.5.1. Remove text and set datatype for monetary columns

In [23]:
# Here, we will strip the sentences and keep only the first element which contains the monetary value we want to keep.
df_cleaning['paid_$'] = df['paid_$'].str.strip()
df_cleaning['paid_$'] = df['paid_$'].str.split().str[0].astype("int32")

df_cleaning['sold_$'] = df['sold_$'].str.strip()
df_cleaning['sold_$'] = df['sold_$'].str.split().str[0].astype("int32")

df_cleaning['profit_$'] = df['profit_$'].str.strip()
df_cleaning['profit_$'] = df['profit_$'].str.split().str[0].astype("int32")

In [24]:
# Rows 16, 19, 31 have text in different monetary columns. We can verify there quickly.
df_cleaning[16:33]

,season_num,episode_num,unique_ep_ID,episode_name,location,air_date,description,paid_$,sold_$,profit_$
16,2,9,209,Animal Instincts,"Orange County, California",2011-05-24,An 1897 Winchester pump-action shotgun and big...,2700,9600,6840
17,2,10,210,Everything's Bigger in Texas,North Texas,2011-05-31,Ton and Allen head 225 miles north of Dallas t...,2975,20570,17595
18,2,11,211,The Smoking Ton,"Worcester, Massachusetts",2011-07-17,"The guys find an 1890 humidor, a gun barrel fo...",1875,10450,8575
19,2,12,212,Top Gun Ton,"Ontario, California",2011-07-19,Ton and Allen buy two units. One contains a co...,1750,10825,8475
20,2,13,213,Beantown Bidders,Boston,2011-07-19,"A 1940s–50s Schwinn tandem bicycle, assorted b...",975,7300,6325
21,2,14,214,Diesel Digs,"Los Angeles, California",2011-08-02,"In an industrial area of Los Angeles, the pair...",1350,8575,7225
22,2,15,215,The Dallas Mavericks,Dallas,2011-08-09,"The guys go to Dallas, where they find a handc...",2150,12125,9975
23,2,16,216,Knuckle Dust,"Hollywood, California",2011-08-16,"The guys sell a 19th-century French ""knuckle d...",1050,13750,12700
24,2,17,217,Half Pipe Dreams,"Long Beach, California",2011-08-23,The guys find a 1970s Z-Flex Jimmy Plumer Pool...,1400,5300,3900
25,2,18,218,Viva la Vegas,"Henderson, Nevada",2011-08-30,The guys head to Las Vegas to acquire a Zig Za...,320,5070,4750


### 3.5.2. Setting Datetime column datatype

In [25]:
df_cleaning['air_date'] = pd.to_datetime(df['air_date'], format='%Y-%m-%d')

In [26]:
df_cleaning[16:20]

,season_num,episode_num,unique_ep_ID,episode_name,location,air_date,description,paid_$,sold_$,profit_$
16,2,9,209,Animal Instincts,"Orange County, California",2011-05-24,An 1897 Winchester pump-action shotgun and big...,2700,9600,6840
17,2,10,210,Everything's Bigger in Texas,North Texas,2011-05-31,Ton and Allen head 225 miles north of Dallas t...,2975,20570,17595
18,2,11,211,The Smoking Ton,"Worcester, Massachusetts",2011-07-17,"The guys find an 1890 humidor, a gun barrel fo...",1875,10450,8575
19,2,12,212,Top Gun Ton,"Ontario, California",2011-07-19,Ton and Allen buy two units. One contains a co...,1750,10825,8475


In [27]:
df_cleaning.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   season_num    107 non-null    int8          
 1   episode_num   107 non-null    int8          
 2   unique_ep_ID  107 non-null    int16         
 3   episode_name  107 non-null    object        
 4   location      106 non-null    object        
 5   air_date      107 non-null    datetime64[ns]
 6   description   107 non-null    object        
 7   paid_$        107 non-null    int32         
 8   sold_$        107 non-null    int32         
 9   profit_$      107 non-null    int32         
dtypes: datetime64[ns](1), int16(1), int32(3), int8(2), object(3)
memory usage: 5.1+ KB


### 3.5.3. Setting Missing Values

In [28]:
# Let's set a description for episode 401
# Source: "https://tv.apple.com/au/episode/once-a-pawn-a-time/umc.cmc.u7kqqkzw16hliw0oa6y8jsd9"
episode_401_description = "Allen and Ton have opened a pawn shop in Los Angeles, but with expenses piling up, even an antique ammo collection and some vintage football helmets may not be enough to keep the boys in business."
episode_401_index = df[df['unique_ep_ID'] == 401].index[0]
df_cleaning.loc[episode_401_index, 'description'] = episode_401_description

In [29]:
df_cleaning[60:63]

,season_num,episode_num,unique_ep_ID,episode_name,location,air_date,description,paid_$,sold_$,profit_$
60,3,26,326,Gold ‘N’ Gloves,"Hermosa Beach, California",2012-09-26,The Auction Hunters team goes toe-to-toe with ...,950,5518,4568
61,4,1,401,Once a Pawn a Time,"Covina, California",2013-01-30,Allen and Ton have opened a pawn shop in Los A...,2000,19000,17000
62,4,2,402,Win Lose Or Joust,"North Hills, Los Angeles",2013-01-30,Ton and Allen race back from an auction to pur...,1050,5500,4450


In [30]:
# Let's set a location for episode 514
### Absent any specific data, "California" is the best guess.
episode_514_location = "California"
episode_514_index = df[df['unique_ep_ID'] == 514].index[0]
df_cleaning.loc[episode_514_index, 'location'] = episode_514_location

In [31]:
df_cleaning[99:102]

,season_num,episode_num,unique_ep_ID,episode_name,location,air_date,description,paid_$,sold_$,profit_$
99,5,13,513,Wreck-Shaw,"Santa Clarita, California",2015-04-18,A weekend auction sees the guys go up against ...,3500,5400,1900
100,5,14,514,Carnies & Armories,California,2015-04-18,A tough group of bidders team up to take down ...,1750,10700,8950
101,5,15,515,The Unsinkable Ton,"Stanton, California",2015-04-25,The guys try to authenticate props from the mo...,2700,5800,3100


In [32]:
df_cleaning

,season_num,episode_num,unique_ep_ID,episode_name,location,air_date,description,paid_$,sold_$,profit_$
0,1,1,101,The Wild West,"San Bernardino, California",2010-11-09,Ton and Allen head to auctions in the desert t...,376,1190,814
1,1,2,102,The Big Score,"Downtown Los Angeles, California",2010-11-09,Ton and Allen bid on units in downtown LA and ...,2025,5850,3825
2,1,3,103,Ton's Got a Gun,"Mission Hills, California",2010-11-16,Allen encounters some old rivals in the Valley...,1900,9450,7550
3,1,4,104,Strat'ed for Cash,"West Adams, Los Angeles",2010-11-23,Allen and Ton venture out to storage auctions ...,975,6200,5225
4,1,5,105,The Real Thing,"Oxnard, California",2010-11-30,"In a small beachtown’s storage unit auction, T...",2450,9400,6950
5,1,6,106,Weapons of Past Destruction,Sunset Boulevard,2010-12-07,Ton and Allen travel to Hollywood and uncover ...,295,1635,1340
6,1,7,107,Gangster Whiskey,"Moreno Valley, California",2010-12-14,"A rare baseball card, a classic jukebox and an...",525,2800,2275
7,1,8,108,Home on the Gun Range,"Corona, California",2010-12-21,A single unit auction yields a 1940s Kissomete...,875,3400,2425
8,2,1,201,Miami Heat,Miami,2011-04-05,"A 1970s NASA countdown clock, a 1960s Delta re...",2800,10640,7840
9,2,2,202,Quadzilla Attacks!,"Playa Vista, Los Angeles",2011-04-05,"An 1880s Palm-Squeezer pistol, a Western Elect...",1250,4990,3740


In [33]:
df_cleaning.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   season_num    107 non-null    int8          
 1   episode_num   107 non-null    int8          
 2   unique_ep_ID  107 non-null    int16         
 3   episode_name  107 non-null    object        
 4   location      107 non-null    object        
 5   air_date      107 non-null    datetime64[ns]
 6   description   107 non-null    object        
 7   paid_$        107 non-null    int32         
 8   sold_$        107 non-null    int32         
 9   profit_$      107 non-null    int32         
dtypes: datetime64[ns](1), int16(1), int32(3), int8(2), object(3)
memory usage: 5.1+ KB


## 3.6. Profit Verification

* Before starting, we will create a new dataframe. Then, I want to confirm whether or not the profit column is correct.
* The equation: Profit = Sold - Paid must hold true for each row. Let's test it.

In [34]:
df_cleaned = df_cleaning.copy()

In [35]:
# Generate a new column showing the calculated profit calculated from paid and sold columns.
df_cleaned['calculated_profit_$'] = (df_cleaning['sold_$'] - df_cleaning['paid_$']).astype('int32')

### 3.6.1. Where is *calculated_profit_S* mismatched from the scraped *profit_S*?

In [37]:
df_cleaned[df_cleaned['profit_$'] != df_cleaned['calculated_profit_$']]

,season_num,episode_num,unique_ep_ID,episode_name,location,air_date,description,paid_$,sold_$,profit_$,calculated_profit_$
7,1,8,108,Home on the Gun Range,"Corona, California",2010-12-21,A single unit auction yields a 1940s Kissomete...,875,3400,2425,2525
16,2,9,209,Animal Instincts,"Orange County, California",2011-05-24,An 1897 Winchester pump-action shotgun and big...,2700,9600,6840,6900
19,2,12,212,Top Gun Ton,"Ontario, California",2011-07-19,Ton and Allen buy two units. One contains a co...,1750,10825,8475,9075
83,4,23,423,"Cashville, Tennessee","Nashville, Tennessee",2014-03-29,"Ton and Allen hit the road for Nashville, wher...",12800,13375,-2425,575


* We can see there are 4 rows where the calculated_profit is different from the scraped profit.
* Let's check the original data to see if there is an explanation, such as in the text we removed.